# Construction d'un système de recommandation

Nous avons décidé d'orienter notre projet sur la recommendation de films.
En effet durant ce confinement, nous avons eu le temps de visionner beaucoup de films,
mais nous nous sommes rendus compte que nous passions quasiment autant de temps
à choisir le film qu'à le regarder. D'où la nécessité de créer un système de re-
commendations afin d'optimiser notre temps de visionnage.
Nous avons chercher une base de données assez exploitable afin de mener à bien
notre projet. Nous nous sommes basés sur la base de données de 'The Movies Dataset'.


In [1]:
import numpy as np
import pandas as pd
import math
import re


## Fetching and cleaning data

Nous utilisons deux tables de données. L'une, *movies_metadata.csv*, contient une liste de films et des informations relativesau genre, date de sortie etc. 

In [2]:
movies = pd.read_csv("movies_metadata.csv")
movies.head()

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [4]:
def filter_correct_id(word):
    if re.fullmatch(r'[0-9]+', word):
        return word
    return "wrong_id"

In [5]:
# don't re-run
movies = movies[~movies.id.duplicated()]
movies.id = movies.id.apply(filter_correct_id)
movies = movies[movies.id != "wrong_id"]
movies.id = movies.id.astype('int64')

In [6]:
movies.id

0           862
1          8844
2         15602
3         31357
4         11862
5           949
6         11860
7         45325
8          9091
9           710
10         9087
11        12110
12        21032
13        10858
14         1408
15          524
16         4584
17            5
18         9273
19        11517
20         8012
21         1710
22         9691
23        12665
24          451
25        16420
26         9263
27        17015
28          902
29        37557
          ...  
45436     45527
45437    455661
45438    327237
45439     84710
45440     39562
45441     14008
45442     44330
45443     49279
45444     44333
45445     49277
45446     49271
45447     44324
45448    122036
45449     14885
45450     49280
45451    106807
45452    276895
45453    404604
45454    420346
45455     67179
45456     84419
45457    390959
45458    289923
45459    222848
45460     30840
45461    439050
45462    111109
45463     67758
45464    227506
45465    461257
Name: id, Length: 45433,

In [7]:
ratings = pd.read_csv("ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
# ne pas re-run !
ratings = ratings.drop(columns=['timestamp'])
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [9]:
ratings[(ratings['userId'] == 1) & (ratings['movieId'] == 31)]

,userId,movieId,rating
0,1,31,2.5


In [10]:
print(min(ratings.rating), max(ratings.rating))
ratings.describe()
ratings.dtypes

0.5 5.0


userId       int64
movieId      int64
rating     float64
dtype: object

In [13]:
nbPers = len(ratings.userId.unique())
nbMovi = len(ratings.movieId.unique())

## User Based

Pour ce système, nous n'avons que besoin des notations des utilisateurs et des titres des films associés.

In [ ]:
movies_title = movies.loc[movies_cleaned['id'].isin(ratings.movieId), ['id', 'title']]

In [14]:
def cor(u, v, df):
    Iu = df.loc[df['userId'] == u, ['movieId']]
    Iv = df.loc[df['userId'] == v, ['movieId']]
    Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')

    su = df.loc[(df['userId'] == u) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    sv = df.loc[(df['userId'] == v) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    su = np.array(su)
    sv = np.array(sv)
    return np.dot(su, sv) / math.sqrt(np.dot(su, su) * np.dot(sv, sv))

In [ ]:
def cor_adj(u, v, df):
    Iu = df.loc[df['userId'] == u, ['movieId']]
    Iv = df.loc[df['userId'] == v, ['movieId']]
    Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')
    
    su = df.loc[(df['userId'] == u) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    sv = df.loc[(df['userId'] == v) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    su = np.array(su)
    sv = np.array(sv)
    
    nb_rat = df.loc[:, ['movieId', 'rating']].groupby(['movieId']).count()
    
    sum_up = 0
    sum_down_u = 0
    sum_down_v = 0
    for movie in Iuv.movieId.unique() :
        suk = Iu.loc[Iu['movieId'] == movie]['rating_norm']
        svk = Iv.loc[Iv['movieId'] == movie]['rating_norm']
        
        sum_up += suk * svk / nb_rat[movie]['rating']
        sum_down_u += suk**2 /  nb_rat[movie]['rating']
        sum_down_v += svk**2 /  nb_rat[movie]['rating']
    return sum_up / sqrt(sum_down_u * sum_down_v)

In [ ]:
def cor_dis(u, v, df):
    beta = 20
    correlation = cor(u, v, df)
    return correlation * min(len(su), beta)/beta

In [ ]:
def cor_dis_adj(u, v, df):
    beta = 20
    correlation = cor_adj(u, v, df)
    return correlation * min(len(su), beta)/beta

In [15]:
def mean_rating(uid):
    n = ratings.loc[ratings['userId'] == uid].count().loc['userId']
    s = ratings.loc[ratings['userId'] == uid].sum().loc['rating']
    return s / n

In [94]:
def normalize(df):
    mean = ratings.loc[:, ['userId']].drop_duplicates()
    mean['mu'] = mean['userId'].map(lambda uid : mean_rating(uid))
    mean = mean.set_index('userId')
    df['rating_norm'] = df[['userId', 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row['userId'])]['mu'], axis=1)

In [96]:
normalize(ratings)

In [ ]:
def correlation_matrix(df, cor_fct=cor):
    correlation = []
    users = df.userId.unique()
    couples = []
    for i in range(len(users)):
        u = users[i]
        print('\nuser : ', u)
        for j in range(i + 1, len(users)):
            v = users[j]
            print('\t', end='')
            print('user : ', v, end='')
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(u, v, df))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    df = pd.DataFrame(correlation, index=index)
    return df

In [ ]:
test = ratings.loc[ratings['userId'] <= 30]
cm = correlation_matrix(ratings)


user :  1
	user :  2	user :  3	user :  4	user :  5	user :  6	user :  7	user :  8	user :  9	user :  10	user :  11	user :  12	user :  13	user :  14	user :  15	user :  16	user :  17	user :  18	user :  19	user :  20	user :  21	user :  22	user :  23	user :  24	user :  25	user :  26	user :  27	user :  28	user :  29	user :  30	user :  31	user :  32	user :  33	user :  34	user :  35	user :  36	user :  37	user :  38	user :  39	user :  40	user :  41	user :  42	user :  43

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


	user :  44	user :  45	user :  46	user :  47	user :  48	user :  49	user :  50	user :  51	user :  52	user :  53	user :  54	user :  55	user :  56	user :  57	user :  58	user :  59	user :  60	user :  61	user :  62	user :  63	user :  64	user :  65	user :  66	user :  67	user :  68	user :  69	user :  70	user :  71	user :  72	user :  73	user :  74	user :  75	user :  76	user :  77	user :  78	user :  79	user :  80	user :  81	user :  82	user :  83	user :  84	user :  85	user :  86	user :  87	user :  88	user :  89	user :  90	user :  91	user :  92	user :  93	user :  94	user :  95	user :  96	user :  97	user :  98	user :  99	user :  100	user :  101	user :  102	user :  103	user :  104	user :  105	user :  106	user :  107	user :  108	user :  109	user :  110	user :  111	user :  112	user :  113	user :  114	user :  115	user :  116	user :  117	user :  118	user :  119	user :  120	user :  121	user :  122	user :  123	user :  124	user :  125	user :  126	user :  127	user :  128	user :  129	user :  130	user :  131

	user :  3	user :  4	user :  5	user :  6	user :  7	user :  8	user :  9	user :  10	user :  11	user :  12	user :  13	user :  14	user :  15	user :  16	user :  17	user :  18	user :  19	user :  20	user :  21	user :  22	user :  23	user :  24	user :  25	user :  26	user :  27	user :  28	user :  29	user :  30	user :  31	user :  32	user :  33	user :  34	user :  35	user :  36	user :  37	user :  38	user :  39	user :  40	user :  41	user :  42	user :  43	user :  44	user :  45	user :  46	user :  47	user :  48	user :  49	user :  50	user :  51	user :  52	user :  53	user :  54	user :  55	user :  56	user :  57	user :  58	user :  59	user :  60	user :  61	user :  62	user :  63	user :  64	user :  65	user :  66	user :  67	user :  68	user :  69	user :  70	user :  71	user :  72	user :  73	user :  74	user :  75	user :  76	user :  77	user :  78	user :  79	user :  80	user :  81	user :  82	user :  83	user :  84	user :  85	user :  86	user :  87	user :  88	user :  89	user :  90	user :  91	user :  92	user :  93	user 

	user :  4	user :  5	user :  6	user :  7	user :  8	user :  9	user :  10	user :  11	user :  12	user :  13	user :  14	user :  15	user :  16	user :  17	user :  18	user :  19	user :  20	user :  21	user :  22	user :  23	user :  24	user :  25	user :  26	user :  27	user :  28	user :  29	user :  30	user :  31	user :  32	user :  33	user :  34	user :  35	user :  36	user :  37	user :  38	user :  39	user :  40	user :  41	user :  42	user :  43	user :  44	user :  45	user :  46	user :  47	user :  48	user :  49	user :  50	user :  51	user :  52	user :  53	user :  54	user :  55	user :  56	user :  57	user :  58	user :  59	user :  60	user :  61	user :  62	user :  63	user :  64	user :  65	user :  66	user :  67	user :  68	user :  69	user :  70	user :  71	user :  72	user :  73	user :  74	user :  75	user :  76	user :  77	user :  78	user :  79	user :  80	user :  81	user :  82	user :  83	user :  84	user :  85	user :  86	user :  87	user :  88	user :  89	user :  90	user :  91	user :  92	user :  93	user :  94	user

...

	user :  181	user :  182	user :  183	user :  184	user :  185	user :  186	user :  187	user :  188	user :  189	user :  190	user :  191	user :  192	user :  193	user :  194	user :  195	user :  196	user :  197	user :  198	user :  199	user :  200	user :  201	user :  202	user :  203	user :  204	user :  205	user :  206	user :  207	user :  208	user :  209	user :  210	user :  211	user :  212	user :  213	user :  214	user :  215	user :  216	user :  217	user :  218	user :  219	user :  220	user :  221	user :  222	user :  223	user :  224	user :  225	user :  226	user :  227	user :  228	user :  229	user :  230	user :  231	user :  232	user :  233	user :  234	user :  235	user :  236	user :  237	user :  238	user :  239	user :  240	user :  241	user :  242	user :  243	user :  244	user :  245	user :  246	user :  247	user :  248	user :  249	user :  250	user :  251	user :  252	user :  253	user :  254	user :  255	user :  256	user :  257	user :  258	user :  259	user :  260	user :  261	user :  262	user :  263	use

	user :  182	user :  183	user :  184	user :  185	user :  186	user :  187	user :  188	user :  189	user :  190	user :  191	user :  192	user :  193	user :  194	user :  195	user :  196	user :  197	user :  198	user :  199	user :  200	user :  201	user :  202	user :  203	user :  204	user :  205	user :  206	user :  207	user :  208	user :  209	user :  210	user :  211	user :  212	user :  213	user :  214	user :  215	user :  216	user :  217	user :  218	user :  219	user :  220	user :  221	user :  222	user :  223	user :  224	user :  225	user :  226	user :  227	user :  228	user :  229	user :  230	user :  231	user :  232	user :  233	user :  234	user :  235	user :  236	user :  237	user :  238	user :  239	user :  240	user :  241	user :  242	user :  243	user :  244	user :  245	user :  246	user :  247	user :  248	user :  249	user :  250	user :  251	user :  252	user :  253	user :  254	user :  255	user :  256	user :  257	user :  258	user :  259	user :  260	user :  261	user :  262	user :  263	user :  264	use

	user :  183	user :  184	user :  185	user :  186	user :  187	user :  188	user :  189	user :  190	user :  191	user :  192	user :  193	user :  194	user :  195	user :  196	user :  197	user :  198	user :  199	user :  200	user :  201	user :  202	user :  203	user :  204	user :  205	user :  206	user :  207	user :  208	user :  209	user :  210	user :  211	user :  212	user :  213	user :  214	user :  215	user :  216	user :  217	user :  218	user :  219	user :  220	user :  221	user :  222	user :  223	user :  224	user :  225	user :  226	user :  227	user :  228	user :  229	user :  230	user :  231	user :  232	user :  233	user :  234	user :  235	user :  236	user :  237	user :  238	user :  239	user :  240	user :  241	user :  242	user :  243	user :  244	user :  245	user :  246	user :  247	user :  248	user :  249	user :  250	user :  251	user :  252	user :  253	user :  254	user :  255	user :  256	user :  257	user :  258	user :  259	user :  260	user :  261	user :  262	user :  263	user :  264	user :  265	use

	user :  184	user :  185	user :  186	user :  187	user :  188	user :  189	user :  190	user :  191	user :  192	user :  193	user :  194	user :  195	user :  196	user :  197	user :  198	user :  199	user :  200	user :  201	user :  202	user :  203	user :  204	user :  205	user :  206	user :  207	user :  208	user :  209	user :  210	user :  211	user :  212	user :  213	user :  214	user :  215	user :  216	user :  217	user :  218	user :  219	user :  220	user :  221	user :  222	user :  223	user :  224	user :  225	user :  226	user :  227	user :  228	user :  229	user :  230	user :  231	user :  232	user :  233	user :  234	user :  235	user :  236	user :  237	user :  238	user :  239	user :  240	user :  241	user :  242	user :  243	user :  244	user :  245	user :  246	user :  247	user :  248	user :  249	user :  250	user :  251	user :  252	user :  253	user :  254	user :  255	user :  256	user :  257	user :  258	user :  259	user :  260	user :  261	user :  262	user :  263	user :  264	user :  265	user :  266	use

	user :  185	user :  186	user :  187	user :  188	user :  189	user :  190	user :  191	user :  192	user :  193	user :  194	user :  195	user :  196	user :  197	user :  198	user :  199	user :  200	user :  201	user :  202	user :  203	user :  204	user :  205	user :  206	user :  207	user :  208	user :  209	user :  210	user :  211	user :  212	user :  213	user :  214	user :  215	user :  216	user :  217	user :  218	user :  219	user :  220	user :  221	user :  222	user :  223	user :  224	user :  225	user :  226	user :  227	user :  228	user :  229	user :  230	user :  231	user :  232	user :  233	user :  234	user :  235	user :  236	user :  237	user :  238	user :  239	user :  240	user :  241	user :  242	user :  243	user :  244	user :  245	user :  246	user :  247	user :  248	user :  249	user :  250	user :  251	user :  252	user :  253	user :  254	user :  255	user :  256	user :  257	user :  258	user :  259	user :  260	user :  261	user :  262	user :  263	user :  264	user :  265	user :  266	user :  267	use

In [ ]:
print(cm)